In [1]:
import numpy as np
import random as r
%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend

In [81]:
# const
BITS = 7
MASK = (1<<BITS) - 1
MIN_FATIGUE=1


def ones(value):
    n = 0
    while value:
        n += 1
        value &= value-1
    return n

def threshold(value, window, adapt):

    mask = (1<<window)-1
    value = ones(value & mask)
    return value >= adapt

class Neuron:
    def __init__(self, min_threshold, uid):
        self.uid = uid
        self.state = 0
        self.min_threshold = min_threshold
        self.threshold = min_threshold
        self.recovery = 0
        self.targets = []
        self.inbox = 0
        
        self.fired = 0
    
    def enqueue(self):
        self.inbox = 1

    def process(self, epoch):
        signal = self.inbox
        self.inbox = 0
        
        def log(msg):
            if self.uid == 0:
                print(msg)
                
        
        self.state = ((self.state << 1) | signal) & MASK


        self.threshold = min(self.threshold, BITS)
        activate = ones(self.state & MASK) >= self.threshold

        if activate:
            for t in self.targets:
                t.enqueue()
            #O[k] = 1
            self.state = 0
            self.threshold += 1
            self.recovery=0
            self.fired = 1

        else:
            self.fired = 0

        if self.recovery>=8:
            self.recovery=0
            self.threshold=max(self.min_threshold, self.threshold-1)    

        if self.threshold:
            self.recovery+=1
        
    pass


class Cleft:
    def __init__(self, neuron):
        self.neuron = neuron
        
        self.inbox = 0
        
    def enqueue(self):
        self.inbox = 1
        
    def process(self, epoch):
        if self.inbox:
            self.neuron.enqueue()
            self.inbox = 0

In [93]:
neurons = [Neuron((i % 2)+1, i) for i in range(1000)]
# connect neurons randomly

clefts = []
for n in neurons:
    targets = r.sample(neurons, 3)
    for t in targets:
        c = Cleft(t)
        n.targets.append(c) 
        clefts.append(c)

fired = 0

for n in neurons[0:10]:
    n.enqueue()
for i in range(100):
    neuron_inbox = 0
    cleft_inbox = 0
    for n in neurons:
        if n.inbox:
            neuron_inbox+=1
        n.process(i)
        if n.fired:
            fired += 1
    for c in clefts:
        if c.inbox:
            cleft_inbox +=1
        c.process(i)
    
        
    print(f"N:{neuron_inbox} C:{cleft_inbox}")
        

N:10 C:15
N:15 C:18
N:18 C:27
N:27 C:30
N:30 C:54
N:53 C:87
N:81 C:93
N:85 C:153
N:145 C:207
N:176 C:252
N:231 C:357
N:302 C:420
N:335 C:423
N:345 C:432
N:344 C:417
N:319 C:363
N:304 C:327
N:274 C:282
N:250 C:276
N:240 C:282
N:244 C:273
N:235 C:255
N:225 C:258
N:227 C:282
N:242 C:249
N:222 C:192
N:171 C:153
N:144 C:108
N:106 C:117
N:112 C:129
N:116 C:102
N:95 C:102
N:99 C:84
N:81 C:99
N:96 C:105
N:102 C:81
N:76 C:87
N:84 C:102
N:99 C:108
N:100 C:87
N:81 C:93
N:88 C:96
N:93 C:150
N:134 C:144
N:132 C:159
N:153 C:201
N:182 C:213
N:191 C:240
N:210 C:255
N:226 C:303
N:257 C:327
N:278 C:327
N:276 C:321
N:279 C:318
N:282 C:315
N:272 C:273
N:245 C:270
N:236 C:249
N:221 C:273
N:236 C:279
N:247 C:285
N:245 C:237
N:214 C:249
N:221 C:228
N:199 C:195
N:178 C:192
N:172 C:210
N:198 C:228
N:203 C:192
N:172 C:171
N:157 C:159
N:146 C:177
N:161 C:159
N:149 C:132
N:123 C:141
N:130 C:141
N:129 C:114
N:109 C:117
N:109 C:129
N:122 C:138
N:129 C:156
N:144 C:135
N:127 C:147
N:132 C:150
N:139 C:141
N:134 C:183
